In [ ]:
#packages for graphics
import matplotlib.pyplot as plt
from matplotlib import colors

#packages for spatial analysis
import geopandas as gpd
import pandas as pd
from pysal.lib import weights
from pysal.explore import esda
import numpy as np

import shapely
from shapely.geometry import Polygon
from shapely.wkt import loads

#read the data
shapefile_path = r".\LSOA_2011_EW_BFC.shp"
geo = gpd.read_file(shapefile_path)

data = pd.read_csv('timeseries.csv')

gdf = geo.merge(data, on='LSOA11CD', how='left')

#Global Moran's I

#generate spatial weighting
w = weights.KNN.from_dataframe(gdf, k=7)
w.transform = 'R'

#excecute Moran function
moran = esda.moran.Moran(gdf['2019'], w)
moran.I.round(3)

#permutations to test significance
moran.p_sim

#Local Moran's I (LISA)

#generate local spatially lagged values
gdf['2019_lag'] = weights.spatial_lag.lag_spatial(w, gdf['2019'])

#standardise the values
def standardize(df, var):
    
    newname = var + '_z'
    df[newname] = (df[var] - df[var].mean()) / df[var].std()

standardize(gdf,'2019')
standardize(gdf,'2019_lag')

#assign 
def rules(row):
    if row['p car_z'] > 0:
        if row['p car_lag_z'] > 0:
            return 'HH'  
        else:
            return 'HL'  
    else:
        if row['p car_lag_z'] > 0:
            return 'LH'  
        else:
            return 'LL'

gdf['quadrant'] = gdf.apply(rules, 1)

#generate LISAs
lisa = esda.moran.Moran_Local(gdf['2019'], w)

#idenfty significant LISAs
gdf['p-sim'] = lisa.p_sim

sig = 1 * (lisa.p_sim < 0.05)
slabels = ['non-sig.', 'significant'] 
labels = [slabels[i] for i in sig]

gdf['sig'] = labels
gdf[['sig','p-sim']].head(10)

#generate labels and assign values to them
qlabels = ['HH', 'LH', 'LL', 'HL']  #pysal scheme is  HH=1, LH=2, LL=3, HL=4
labels = [qlabels[i-1] for i in lisa.q]  #list substituting 1-4 with HH-HL
labels[1:10]

gdf['qlabels'] = labels
[(qlabel, (gdf['qlabels']==qlabel).sum()) for qlabel in qlabels]

hotspot = 1 * (sig * lisa.q==1)
coldspot = 3 * (sig * lisa.q==3)
doughnut = 2 * (sig * lisa.q==2)
diamond = 4 * (sig * lisa.q==4)
spots = hotspot + coldspot + doughnut + diamond
spot_labels = [ 'Not Significant', 'High-High', 'Low-High', 'Low-Low', 'High-Low']
labels = [spot_labels[i] for i in spots]

gdf['slabels'] = labels
[(spot_label, (gdf['slabels']==spot_label).sum()) for spot_label in spot_labels]

#create LISA map
fig, ax = plt.subplots(1, figsize=(11,15))
ax.axis('off')

sigcolors = colors.ListedColormap([ 'red', 'lightcoral', 'skyblue', 'blue', 'lightgrey'])

gdf.plot(column='slabels', categorical=True,
         k=2, cmap=sigcolors, linewidth=0.1, edgecolor='white', 
         legend=True, legend_kwds={"title":"LISA (Moran's I)","loc": 'lower right'},
         ax=ax)

plt.title('LISA with respect to Median Electricity Consumption, 2019', fontsize=24) 

ax.annotate("Source: BEIS (2019)",
             xy=(0.1, 0.1), xycoords='figure fraction', 
             horizontalalignment='left', verticalalignment='top', 
             fontsize=16, color='#555555') 

fig.savefig('LISA_elec.png', dpi=300, bbox_inches='tight', facecolor='white')

plt.show()